In [1]:
from sklearn.preprocessing import MultiLabelBinarizer
from ast import literal_eval
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from nltk.stem.snowball import SnowballStemmer
import gensim
import scipy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [2]:
train = pd.read_csv('train.tsv',sep='\t')
test = pd.read_csv('test.tsv',sep='\t')

In [3]:
train['Product Long Description'] = train['Product Long Description'].fillna('')
train['Product Short Description'] = train['Product Short Description'].fillna('')
train['Short Description'] = train['Short Description'].fillna('')
train['Recommended Location'] = train['Recommended Location'].fillna('')
train['Recommended Use'] = train['Recommended Use'].fillna('')

test['Product Long Description'] = test['Product Long Description'].fillna('')
test['Product Short Description'] = test['Product Short Description'].fillna('')
test['Short Description'] = test['Short Description'].fillna('')
test['Recommended Location'] = test['Recommended Location'].fillna('')
test['Recommended Use'] = test['Recommended Use'].fillna('')

In [4]:
for i,row in train.iterrows():
    if type(row['Product Name']) is not float:
        complete_info = row['Product Name'] + ' ' + row['Short Description'] + ' ' + row['Product Short Description'] + ' ' + row['Product Long Description'] + ' ' + row['Recommended Location'] + ' ' + row['Recommended Use']
        train.set_value(i,'Product_Info',complete_info)
        
for i,row in test.iterrows():
    if type(row['Product Name']) is not float:
        complete_info = row['Product Name'] + ' ' + row['Short Description'] + ' ' + row['Product Short Description'] + ' ' + row['Product Long Description'] + ' ' + row['Recommended Location'] + ' ' + row['Recommended Use']
        test.set_value(i,'Product_Info',complete_info)

In [5]:
train.drop(['actual_color','item_id','Seller','Color',
            'Product Long Description','Product Name','Product Short Description','Recommended Room',
            'Short Description'], axis = 1, inplace=True)
test.drop(['actual_color','Seller','Color',
            'Product Long Description','Product Name','Product Short Description','Recommended Room',
            'Short Description'], axis = 1, inplace=True)

In [6]:
train = train[~train.Product_Info.isnull()]

In [7]:
bad_input = []
for i, row in test.iterrows():
    if type(row.Product_Info) is float:
        bad_input.append(row.item_id)

In [8]:
itemid_test = test.item_id

In [9]:
test.drop(['item_id'], axis = 1, inplace=True)

In [10]:
test = test[~test.Product_Info.isnull()]

In [11]:
mlb = MultiLabelBinarizer()
train['tag'] = train.tag.apply(lambda x: [literal_eval(x)][0])
Y_train = mlb.fit_transform(train.tag)

In [12]:
for i,row in train.iterrows():
    train.set_value(i,'Product_Class','Other')
    if type(row.Actors) is not float or type(row['MPAA Rating']) is not float or type(row['Aspect Ratio']) is not float:
        train.set_value(i,'Product_Class','Movie')
    elif type(row['ISBN']) is float and type(row['Synopsis']) is not float:
        train.set_value(i,'Product_Class','Movie')
    elif type(row['Actual Color']) is not float:
        train.set_value(i,'Product_Class','Movie')
    elif type(row['Artist ID']) is not float or type(row['Genre ID']) is not float:
        train.set_value(i,'Product_Class','Music')
    elif type(row['ISBN']) is float and type(row['Publisher']) is not float:
        train.set_value(i,'Product_Class','VideoGame')
    if type(row['ISBN']) is not float and type(row['Synopsis']) is not float:
        train.set_value(i,'Product_Class','Book')
    elif type(row['ISBN']) is not float or type(row['Literary Genre']) is not float:
        train.set_value(i,'Product_Class','Book')
        
for i,row in test.iterrows():
    test.set_value(i,'Product_Class','Other')
    if type(row.Actors) is not float or type(row['MPAA Rating']) is not float or type(row['Aspect Ratio']) is not float:
        test.set_value(i,'Product_Class','Movie')
    elif type(row['ISBN']) is float and type(row['Synopsis']) is not float:
        test.set_value(i,'Product_Class','Movie')
    elif type(row['Actual Color']) is not float:
        test.set_value(i,'Product_Class','Movie')
    elif type(row['Artist ID']) is not float or type(row['Genre ID']) is not float:
        test.set_value(i,'Product_Class','Music')
    elif type(row['ISBN']) is float and type(row['Publisher']) is not float:
        test.set_value(i,'Product_Class','VideoGame')
    if type(row['ISBN']) is not float and type(row['Synopsis']) is not float:
        test.set_value(i,'Product_Class','Book')
    elif type(row['ISBN']) is not float or type(row['Literary Genre']) is not float:
        test.set_value(i,'Product_Class','Book')

In [13]:

for i, row in train.iterrows():
    if type(row.Actors) is not float:
        train.set_value(i,'Actors',1)
    else:
        train.set_value(i,'Actors',0)
    if type(row['MPAA Rating']) is not float:
        train.set_value(i,'MPAA Rating',1)
    else:
        train.set_value(i,'MPAA Rating',0)
    if type(row['Aspect Ratio']) is not float:
        train.set_value(i,'Aspect Ratio',1)
    else:
        train.set_value(i,'Aspect Ratio',0)
    if type(row.ISBN) is not float:
        train.set_value(i,'ISBN',1)
    else:
        train.set_value(i,'ISBN',0)
    if type(row.Synopsis) is not float:
        train.set_value(i,'Synopsis',1)
    else:
        train.set_value(i,'Synopsis',0)
    if type(row['Artist ID']) is not float:
        train.set_value(i,'Artist ID',1)
    else:
        train.set_value(i,'Artist ID',0)
    if type(row.Publisher) is not float:
        train.set_value(i,'Publisher',1)
    else:
        train.set_value(i,'Publisher',0)
    if type(row['Genre ID']) is not float:
        train.set_value(i,'Genre ID',1)
    else:
        train.set_value(i,'Genre ID',0)
    if type(row['Literary Genre']) is not float:
        train.set_value(i,'Literary Genre',1)
    else:
        train.set_value(i,'Literary Genre',0)
    if type(row['Actual Color']) is not float:
        train.set_value(i,'Actual Color',1)
    else:
        train.set_value(i,'Actual Color',0)
        
for i, row in test.iterrows():
    if type(row.Actors) is not float:
        test.set_value(i,'Actors',1)
    else:
        test.set_value(i,'Actors',0)
    if type(row['MPAA Rating']) is not float:
        test.set_value(i,'MPAA Rating',1)
    else:
        test.set_value(i,'MPAA Rating',0)
    if type(row['Aspect Ratio']) is not float:
        test.set_value(i,'Aspect Ratio',1)
    else:
        test.set_value(i,'Aspect Ratio',0)
    if type(row.ISBN) is not float:
        test.set_value(i,'ISBN',1)
    else:
        test.set_value(i,'ISBN',0)
    if type(row.Synopsis) is not float:
        test.set_value(i,'Synopsis',1)
    else:
        test.set_value(i,'Synopsis',0)
    if type(row['Artist ID']) is not float:
        test.set_value(i,'Artist ID',1)
    else:
        test.set_value(i,'Artist ID',0)
    if type(row.Publisher) is not float:
        test.set_value(i,'Publisher',1)
    else:
        test.set_value(i,'Publisher',0)
    if type(row['Genre ID']) is not float:
        test.set_value(i,'Genre ID',1)
    else:
        test.set_value(i,'Genre ID',0)
    if type(row['Literary Genre']) is not float:
        test.set_value(i,'Literary Genre',1)
    else:
        test.set_value(i,'Literary Genre',0)
    if type(row['Actual Color']) is not float:
        test.set_value(i,'Actual Color',1)
    else:
        test.set_value(i,'Actual Color',0)

In [14]:
train.drop(['tag'], axis = 1, inplace=True)

In [15]:
train.drop(['Recommended Use'], axis = 1, inplace=True)
test.drop(['Recommended Use'], axis = 1, inplace=True)

In [16]:
#train.drop(['Actors','MPAA Rating','Aspect Ratio','ISBN','Synopsis',
#            'Artist ID','Publisher','Genre ID','Literary Genre','tag'], axis = 1, inplace=True)
#test.drop(['Actors','MPAA Rating','Aspect Ratio','ISBN','Synopsis',
#            'Artist ID','Publisher','Genre ID','Literary Genre'], axis = 1, inplace=True)

In [17]:
for i, row in train.iterrows():
    if row.Product_Class in ['Movie','Book','VideoGame']:
        train.set_value(i,'Recommended Location',1)
    else:
        train.set_value(i,'Recommended Location',0)
for i, row in test.iterrows():
    if row.Product_Class in ['Movie','Book','VideoGame']:
        test.set_value(i,'Recommended Location',1)
    else:
        test.set_value(i,'Recommended Location',0)

In [18]:
for i, row in train.iterrows():
    soup = BeautifulSoup(row.Product_Info, 'html.parser')
    train.set_value(i,'Product_Info',soup.get_text(separator=' '))

for i, row in test.iterrows():
    soup = BeautifulSoup(row.Product_Info, 'html.parser')
    test.set_value(i,'Product_Info',soup.get_text(separator=' '))

In [19]:
train['Product_Info'] = train['Product_Info'].str.lower()
train.Product_Info = train.Product_Info.str.replace(r'[0-9]',' ')
train.Product_Info = train.Product_Info.str.replace(r'https?.*?(\s|\n)',' ')
train.Product_Info = train.Product_Info.str.replace(r'[\w\.-]+@[\w\.-]+',' ')
train.Product_Info = train.Product_Info.str.replace(r'\.|\-|\+|\\|\/|\;|\:|\!|\_|\@|\#|\$|\%|\^|\&|\*|\(|\)|\{|\}|\[|\]|\||\<|\>|\"|\'|\,|\?',' ')
train.Product_Info = train.Product_Info.str.replace(r'\s[a-zA-Z]{2}\s|\s[a-zA-Z]{1}\s',' ')

test['Product_Info'] = test['Product_Info'].str.lower()
test.Product_Info = test.Product_Info.str.replace(r'[0-9]',' ')
test.Product_Info = test.Product_Info.str.replace(r'https?.*?(\s|\n)',' ')
test.Product_Info = test.Product_Info.str.replace(r'[\w\.-]+@[\w\.-]+',' ')
test.Product_Info = test.Product_Info.str.replace(r'\.|\-|\+|\\|\/|\;|\:|\!|\_|\@|\#|\$|\%|\^|\&|\*|\(|\)|\{|\}|\[|\]|\||\<|\>|\"|\'|\,|\?',' ')
test.Product_Info = test.Product_Info.str.replace(r'\s[a-zA-Z]{2}\s|\s[a-zA-Z]{1}\s',' ')

In [20]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")

In [21]:
for i, row in train.iterrows():
    text = row.Product_Info.split()
    text = [word for word in text if word not in stopwords]
    #text = [stemmer.stem(t) for t in text]
    train.set_value(i,'Product_Info',' '.join(text))

for i, row in test.iterrows():
    text = row.Product_Info.split()
    text = [word for word in text if word not in stopwords]
    #text = [stemmer.stem(t) for t in text]
    test.set_value(i,'Product_Info',' '.join(text))

In [22]:
#train.Product_Info = train.Product_Info.str.replace(r'\s[a-zA-Z]{2}\s|\s[a-zA-Z]{1}\s',' ')
#test.Product_Info = test.Product_Info.str.replace(r'\s[a-zA-Z]{2}\s|\s[a-zA-Z]{1}\s',' ')

In [23]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0, ngram_range= (1,2), max_features = 50000)

In [24]:
tfidf = tfidf_vectorizer.fit_transform(train.Product_Info)
tfidf_test = tfidf_vectorizer.transform(test.Product_Info)

In [25]:
dftr = pd.get_dummies(train[['Product_Class','Item Class ID']])
dfte = pd.get_dummies(test[['Product_Class','Item Class ID']])
dfte = dfte.reindex(columns = dftr.columns, fill_value=0)

In [26]:
train.drop(['Product_Class','Item Class ID','Product_Info'], axis = 1, inplace=True)
test.drop(['Product_Class','Item Class ID','Product_Info'], axis = 1, inplace=True)
#train.drop(['Product_Class','Item Class ID'], axis = 1, inplace=True)
#test.drop(['Product_Class','Item Class ID'], axis = 1, inplace=True)

In [27]:
train = train.apply(lambda x: pd.to_numeric(x))
test = test.apply(lambda x: pd.to_numeric(x))

In [28]:
dftr = scipy.sparse.csr_matrix(dftr.values)
dfte = scipy.sparse.csr_matrix(dfte.values)
train = scipy.sparse.csr_matrix(train.values)
test = scipy.sparse.csr_matrix(test.values)

In [29]:
X = scipy.sparse.hstack([tfidf, dftr,train])
X_te = scipy.sparse.hstack([tfidf_test, dfte,test])

In [30]:
X

<10584x50039 sparse matrix of type '<type 'numpy.float64'>'
	with 1159835 stored elements in COOrdinate format>

In [31]:
classif = OneVsRestClassifier(SVC(kernel='linear'))
classif.fit(X, Y_train)

OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=1)

In [32]:
out = classif.predict(X_te)

In [33]:
mlbout = mlb.inverse_transform(out)

In [34]:
bad_input

[12928L, 14691L, 16725L, 18767L, 20014L, 20802L]

In [35]:
final = []
k = 0
for i in range(len(itemid_test)):  
    if itemid_test[i] in bad_input:
        final.append(tuple([itemid_test[i],list()]))
        k = k+1
    else:
        final.append(tuple([itemid_test[i],list(mlbout[i-k])]))

In [36]:
output = pd.DataFrame(final,columns=['item_id','tag'])

In [37]:
output.to_csv('tags.tsv',sep='\t',index=False)

In [ ]:
#set(test['Item Class ID']).difference(set(train['Item Class ID']))

In [ ]:
#train[train['Item Class ID'].isnull() & ~train['Product Name'].isnull()]

In [ ]:
#for i,row in train[train['Item Class ID'].isnull() & ~train['Product Name'].isnull() & ~train['ISBN'].isnull()].iterrows():
#    train[train['Item Class ID'].isnull() & ~train['Product Name'].isnull() & ~train['ISBN'].isnull()].set_value(i,'Item Class ID',3)

In [ ]:
#train[train['Item Class ID'].isnull() & ~train['Product Name'].isnull() & ~train['ISBN'].isnull()]

In [ ]:
#train[~train['ISBN'].isnull()]['Item Class ID'].value_counts()